In [12]:
import boto3
import sagemaker
import csv
import pandas as pd

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket = sagemaker_session.default_bucket()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name="sagemaker", region_name=region)

In [16]:
!aws s3 cp --recursive s3://aurelia-resort-data/model_train ./data_csv

download: s3://aurelia-resort-data/model_train/data_csv/test.csv to data_csv/data_csv/test.csv
download: s3://aurelia-resort-data/model_train/data_csv/train.csv to data_csv/data_csv/train.csv
download failed: s3://aurelia-resort-data/model_train/deepAR/output/deepar-hotel-2023-04-02-22-43-53-696/debug-output/training_job_end.ts to data_csv/deepAR/output/deepar-hotel-2023-04-02-22-43-53-696/debug-output/training_job_end.ts An error occurred (AccessDenied) when calling the GetObject operation: The ciphertext refers to a customer master key that does not exist, does not exist in this region, or you are not allowed to access.
download failed: s3://aurelia-resort-data/model_train/deepAR/output/deepar-hotel-2023-04-02-22-43-53-696/profiler-output/system/incremental/2023040222/1680475620.algo-1.json to data_csv/deepAR/output/deepar-hotel-2023-04-02-22-43-53-696/profiler-output/system/incremental/2023040222/1680475620.algo-1.json An error occurred (AccessDenied) when calling the GetObject oper

In [19]:
train = pd.read_csv("data_csv/data_csv/train.csv")
test = pd.read_csv("data_csv/data_csv/test.csv")

In [21]:
autopilot_train_path = "./traindataautopilot.csv"
train.to_csv(autopilot_train_path, index=False, header=True)

In [23]:
train_s3_prefix = "data"
autopilot_train_s3_uri = sagemaker_session.upload_data(path=autopilot_train_path, key_prefix=train_s3_prefix)
autopilot_train_s3_uri

's3://sagemaker-us-east-1-478947633708/data/traindataautopilot.csv'

In [24]:
!aws s3 ls $autopilot_train_s3_uri

2023-04-03 01:43:58       2365 traindataautopilot.csv


In [ ]:
%store autopilot_train_s3_uri

Stored 'autopilot_train_s3_uri' (str)


In [ ]:
%store


Stored variables and their in-db values:
autopilot_train_s3_uri                  -> 's3://sagemaker-us-east-1-478947633708/data/traind
bookings_grouped                        ->          Date  total_bookings
0  2014-10-01       
cancellations_grouped                   ->          Date  cancellations
0  2014-10-01        
finaldf                                 -> Empty DataFrame
Columns: [Date, TotalIncome, Total
income_grouped                          ->          Date  income_by_year
0  1953-01-01       
income_value_grouped                    ->          Date  income_total
0  1953-01-01         
listings_clean                          ->    month_year  num_listings
0     2022-12         
listings_date                           ->          Date  num_listings
0  2022-12-01         
reviews_clean                           ->     month_year  num_stays
0      2009-05          
reviews_date                            ->           Date  num_stays
0   2009-05-01          
setup_dependencies_

In [ ]:

try:
    autopilot_train_s3_uri
    print("[OK]")
except NameError:
    print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] PLEASE RUN THE PREVIOUS 01_PREPARE_DATASET_AUTOPILOT NOTEBOOK.")
    print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")

[OK]


In [ ]:
if not autopilot_train_s3_uri:
    print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] PLEASE RUN THE PREVIOUS 01_PREPARE_DATASET_AUTOPILOT NOTEBOOK.")
    print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
else:
    print("[OK]")

[OK]


In [ ]:
print(autopilot_train_s3_uri)

s3://sagemaker-us-east-1-478947633708/data/traindataautopilot.csv


In [ ]:
!aws s3 ls $autopilot_train_s3_uri

2023-04-03 01:43:58       2365 traindataautopilot.csv


In [32]:
import csv

df = pd.read_csv("traindataautopilot.csv")
df.head()

,Date,TotalAirlineTripstoDC,TotalAirlinePassengerstoDC,TotalAmericanTravelers,PercentofAmericanswhoTraveled,TotalTripsbyAmericans,num_stays,income_total
0,2019-01-01,435,1326271.0,2.652241e+08,81.442421,1.137042e+09,2899,14791.2
1,2019-02-01,385,1261292.0,2.662408e+08,81.774326,1.148667e+09,2639,14835.3
2,2019-03-01,422,1668002.0,2.650919e+08,81.799085,1.306565e+09,5226,14843.9
3,2019-04-01,427,1681832.0,2.640626e+08,81.354347,1.446882e+09,5608,14811.8
4,2019-05-01,439,1766148.0,2.661340e+08,81.884670,1.415057e+09,6110,14814.7


In [33]:
prefix_model_output = "models/autopilot"

model_output_s3_uri = "s3://{}/{}".format(bucket, prefix_model_output)

print(model_output_s3_uri)

s3://sagemaker-us-east-1-478947633708/models/autopilot


In [34]:
max_candidates = 3

job_config = {
    "CompletionCriteria": {
        "MaxRuntimePerTrainingJobInSeconds": 900,
        "MaxCandidates": max_candidates,
        "MaxAutoMLJobRuntimeInSeconds": 5400,
    },
}

input_data_config = [
    {
        "DataSource": {"S3DataSource": {"S3DataType": "S3Prefix", "S3Uri": "{}".format(autopilot_train_s3_uri)}},
        "TargetAttributeName": "star_rating",
    }
]

output_data_config = {"S3OutputPath": "{}".format(model_output_s3_uri)}

In [35]:
existing_jobs_response = sm.list_auto_ml_jobs()

In [36]:
num_existing_jobs = 0
running_jobs = 0

if "AutoMLJobSummaries" in existing_jobs_response.keys():
    job_list = existing_jobs_response["AutoMLJobSummaries"]
    num_existing_jobs = len(job_list)
    # print('[INFO] You already created {} Autopilot job(s) in this account.'.format(num_existing_jobs))
    for j in job_list:
        if "AutoMLJobStatus" in j.keys():
            if j["AutoMLJobStatus"] == "InProgress":
                running_jobs = running_jobs + 1
    print("[INFO] You have {} Autopilot job(s) currently running << Should be 0 jobs.".format(running_jobs))
else:
    print("[OK] Please continue.")

[INFO] You have 0 Autopilot job(s) currently running << Should be 0 jobs.


In [37]:
from time import gmtime, strftime, sleep

In [38]:
%store -r auto_ml_job_name

try:
    auto_ml_job_name
except NameError:
    timestamp_suffix = strftime("%d-%H-%M-%S", gmtime())
    auto_ml_job_name = "automl-dm-" + timestamp_suffix
    print("Created AutoMLJobName: " + auto_ml_job_name)

no stored variable or alias auto_ml_job_name
Created AutoMLJobName: automl-dm-03-01-52-28


In [39]:
print(auto_ml_job_name)

automl-dm-03-01-52-28


In [40]:
%store auto_ml_job_name

Stored 'auto_ml_job_name' (str)


In [41]:
max_running_jobs = 1

if running_jobs < max_running_jobs:  # Limiting to max. 1 Jobs
    try:
        sm.create_auto_ml_job(
            AutoMLJobName=auto_ml_job_name,
            InputDataConfig=input_data_config,
            OutputDataConfig=output_data_config,
            AutoMLJobConfig=job_config,
            RoleArn=role,
        )
        print("[OK] Autopilot Job {} created.".format(auto_ml_job_name))
        running_jobs = running_jobs + 1
    except:
        print(
            "[INFO] You have already launched an Autopilot job. Please continue see the output of this job.".format(
                running_jobs
            )
        )
else:
    print(
        "[INFO] You have already launched {} Autopilot running job(s). Please continue see the output of the running job.".format(
            running_jobs
        )
    )

[INFO] You have already launched an Autopilot job. Please continue see the output of this job.
